In [19]:
# whiteout_survival_battle_simulator.py
"""
Whiteout Survival Battle Simulator (No Heroes)
=============================================

Tier‑aware sandbox for troop theory‑crafting in *Whiteout Survival*.

Version v0.7  (ASCII‑only comments)
-----------------------------------
* Replaced all fancy Unicode characters (em dashes, arrows) with plain ASCII.
* Updated base template: Archer now has lethality 5 (was 4).
* Previous debug and refactor improvements remain unchanged.
"""

import logging
from dataclasses import dataclass, field
from typing import Dict, Tuple

MAX_TURNS: int = 50  # failsafe against infinite loops
logger = logging.getLogger("whiteout.battle")

# ----------------------------------------------------------------------
# Tier templates (base stats before adding tier points)
# order: (attack, defense, health, lethality)
# ----------------------------------------------------------------------
TEMPLATE: Dict[str, Tuple[int, int, int, int]] = {
    "Infantry": (0, 3, 5, 0),
    "Cavalry":  (3, 1, 1, 4),
    "Archer":   (4, 0, 0, 5),
}


def tier_stats(tier: int) -> Dict[str, "TroopStats"]:
    """Generate stats for the given tier (tier points are added to each base stat)."""
    out: Dict[str, TroopStats] = {}
    for name, (atk_b, def_b, hp_b, leth_b) in TEMPLATE.items():
        out[name] = TroopStats(
            name=name,
            attack=atk_b + tier,
            defense=def_b + tier,
            health=hp_b + tier,
            lethality=leth_b + tier,
        )
    return out

# ----------------------------------------------------------------------
# Data classes
# ----------------------------------------------------------------------
@dataclass
class TroopStats:
    name: str
    attack: float
    defense: float
    health: float
    lethality: float


@dataclass
class Buffs:
    attack_pct: float = 0.0
    defense_pct: float = 0.0
    health_pct: float = 0.0
    lethality_pct: float = 0.0

    def apply(self, base: "TroopStats") -> "TroopStats":
        """Return a new TroopStats with percentage buffs applied."""
        scale = lambda pct: 1.0 + pct / 100.0
        return TroopStats(
            name=base.name,
            attack=base.attack * scale(self.attack_pct),
            defense=base.defense * scale(self.defense_pct),
            health=base.health * scale(self.health_pct),
            lethality=base.lethality * scale(self.lethality_pct),
        )


@dataclass
class TroopGroup:
    base: TroopStats
    count: int
    buffs: Buffs

    eff: TroopStats = field(init=False)
    defense_pool: float = field(init=False)
    health_pool: float = field(init=False)
    injured: int = field(default=0, init=False)

    def __post_init__(self) -> None:
        self.eff = self.buffs.apply(self.base)
        self.defense_pool = self.count * self.eff.defense
        self.health_pool = self.count * self.eff.health

    # ------------------------------------------------------------------
    def receive_damage(self, atk_dmg: float, leth_dmg: float) -> Tuple[float, float]:
        """Apply attack and lethality damage to this troop line.

        Returns the remaining (atk, leth) spillover values.
        """
        if self.active_count == 0 or (atk_dmg <= 0 and leth_dmg <= 0):
            return atk_dmg, leth_dmg

        prev_active = self.active_count

        # Attack first removes defense
        absorbed_def = min(self.defense_pool, atk_dmg)
        self.defense_pool -= absorbed_def
        atk_after_def = atk_dmg - absorbed_def

        # Remaining attack plus full lethality can damage health
        hp_damage_cap = atk_after_def + leth_dmg
        absorbed_hp = min(self.health_pool, hp_damage_cap)
        self.health_pool -= absorbed_hp

        # Injuries are difference in active count
        self.injured += prev_active - self.active_count

        # Debug output
        logger.debug(
            "%s | DEF %d HP %d | dmg_to_HP atk %d leth %d",
            self.base.name,
            round(self.defense_pool),
            round(self.health_pool),
            round(min(atk_after_def, absorbed_hp)),
            round(absorbed_hp - min(atk_after_def, absorbed_hp)),
        )

        # Calculate leftover portions for next troop line
        atk_used = min(atk_after_def, absorbed_hp)
        leth_used = absorbed_hp - atk_used
        return atk_after_def - atk_used, leth_dmg - leth_used

    # ------------------------------------------------------------------
    @property
    def active_count(self) -> int:
        """Return current number of surviving soldiers in this group."""
        return 0 if self.eff.health == 0 else max(0, int(self.health_pool // self.eff.health))


@dataclass
class Army:
    troops: Dict[str, TroopGroup]

    def total_attack(self) -> float:
        return sum(t.active_count * t.eff.attack for t in self.troops.values())

    def total_lethality(self) -> float:
        return sum(t.active_count * t.eff.lethality for t in self.troops.values())

    def has_active(self) -> bool:
        return any(t.active_count > 0 for t in self.troops.values())

    def injured_report(self) -> Dict[str, int]:
        return {k: g.injured for k, g in self.troops.items()}

# ----------------------------------------------------------------------
# Battle loop
# ----------------------------------------------------------------------

def simulate(attacker: "Army", defender: "Army", *, max_turns: int = MAX_TURNS):
    """Run the battle simulation and return summary statistics.

    The function now logs **total injured / total troops** for each side at the
    end (INFO level).  Per‑troop breakdowns remain available at DEBUG level
    inside `receive_damage()`.
    """
    order = ("Infantry", "Cavalry", "Archer")
    turn = 0

    while attacker.has_active() and defender.has_active() and turn < max_turns:
        turn += 1

        atk_pool = (attacker.total_attack(), attacker.total_lethality())
        def_pool = (defender.total_attack(), defender.total_lethality())

        logger.debug(
            "Turn %d | attacker A %.0f L %.0f | defender A %.0f L %.0f",
            turn, *atk_pool, *def_pool
        )

        for (src, tgt, pool) in (
            (attacker, defender, atk_pool),
            (defender, attacker, def_pool),
        ):
            spill_atk, spill_leth = pool
            for troop in order:
                spill_atk, spill_leth = tgt.troops[troop].receive_damage(spill_atk, spill_leth)
                if spill_atk <= 0 and spill_leth <= 0:
                    break

    # -------- Final summary (totals) ---------------------------------
    atk_total = sum(g.count for g in attacker.troops.values())
    def_total = sum(g.count for g in defender.troops.values())
    atk_inj_total = sum(g.injured for g in attacker.troops.values())
    def_inj_total = sum(g.injured for g in defender.troops.values())

    logger.info(
        "Battle ended in %d turns | Attacker injured %d / %d (%.1f%%) | Defender injured %d / %d (%.1f%%)",
        turn,
        atk_inj_total,
        atk_total,
        100 * atk_inj_total / atk_total if atk_total else 0,
        def_inj_total,
        def_total,
        100 * def_inj_total / def_total if def_total else 0,
    )

    return turn, attacker.injured_report(), defender.injured_report()

# ----------------------------------------------------------------------
# ----------------------------------------------------------------------
# Manual test demonstrating BUFFS (only runs when executed directly)
# ----------------------------------------------------------------------
if __name__ == "__main__":
    logging.basicConfig(format="%(levelname)s | %(message)s", level=logging.DEBUG)

    troop_stats = tier_stats(8)
    print(troop_stats["Infantry"])
    print(troop_stats["Archer"])

    atk_counts = {"Infantry": 160000, "Cavalry": 140000, "Archer": 138000}
    def_counts = {"Infantry": 80000, "Cavalry": 92000, "Archer": 92000}

    # ------------------- Buff definitions -------------------
    # Defender gets a flat 300 % to every stat
    def_buffs = {k: Buffs(attack_pct=300, defense_pct=300, health_pct=300, lethality_pct=300) for k in troop_stats}

    # Attacker stats
    atk_buffs = {
        "Infantry": Buffs(attack_pct=180, defense_pct=180, health_pct=150, lethality_pct=109),
        "Cavalry":  Buffs(attack_pct=200, defense_pct=200, health_pct=100, lethality_pct=140),
        "Archer":   Buffs(attack_pct=140, defense_pct=160, health_pct=180, lethality_pct=200),
    }

    # Skills
    # * 0.84 defender attack
    # * 1.25 attacker health
    # * 1.20 attacker lethality
    # * 1.20 attacker lethality
    # * 1.06 attacker defense
    # * 1.09 attacker health

    # ------------------- Global skill modifiers -------------------
    # Multiplicative factors that apply to *all* troops on a side
    atk_global = {"health": 1.25 * 1.09 * 2 * 2, # howard, saul
                  "defense": 1.06, # saul
                  "lethality": 1.20 * 1.20, # saul, jabel
                  "attack": 1.00}

    def_global = {"attack": 0.84, # howard
                  "defense": 1.00,
                  "health": 1.00,
                  "lethality": 1.00}

    # Helper to merge global modifiers into existing Buffs objects
    def apply_globals(buff: Buffs, g: Dict[str, float]) -> Buffs:
        return Buffs(
            attack_pct   =( (1 + buff.attack_pct/100)   * g["attack"]   - 1) * 100,
            defense_pct  =( (1 + buff.defense_pct/100)  * g.get("defense",1.0) - 1) * 100,
            health_pct   =( (1 + buff.health_pct/100)   * g.get("health",1.0)  - 1) * 100,
            lethality_pct=( (1 + buff.lethality_pct/100)* g.get("lethality",1.0)- 1) * 100,
        )

    atk_buffs = {k: apply_globals(b, atk_global) for k, b in atk_buffs.items()}
    def_buffs = {k: apply_globals(b, def_global) for k, b in def_buffs.items()}

    attacker = Army({k: TroopGroup(troop_stats[k], atk_counts[k], atk_buffs[k]) for k in troop_stats})
    defender = Army({k: TroopGroup(troop_stats[k], def_counts[k], def_buffs[k]) for k in troop_stats})

    turns, atk_inj, def_inj = simulate(attacker, defender)
    logger.debug("Attacker injured -> %s", atk_inj)
    logger.debug("Defender injured -> %s", def_inj)


DEBUG | Turn 1 | attacker A 12178400 L 17408448 | defender A 9260160 L 11760000
DEBUG | Infantry | DEF 0 HP 0 | dmg_to_HP atk 4160000 leth 0
DEBUG | Cavalry | DEF 0 HP 0 | dmg_to_HP atk 1186400 leth 2125600
DEBUG | Archer | DEF 2944000 HP 0 | dmg_to_HP atk 0 leth 2944000
DEBUG | Infantry | DEF 0 HP 12543520 | dmg_to_HP atk 4036480 leth 11760000
INFO | Battle ended in 1 turns | Attacker injured 89183 / 438000 (20.4%) | Defender injured 264000 / 264000 (100.0%)
DEBUG | Attacker injured -> {'Infantry': 89183, 'Cavalry': 0, 'Archer': 0}
DEBUG | Defender injured -> {'Infantry': 80000, 'Cavalry': 92000, 'Archer': 92000}


TroopStats(name='Infantry', attack=8, defense=11, health=13, lethality=8)
TroopStats(name='Archer', attack=12, defense=8, health=8, lethality=13)
